<a href="https://colab.research.google.com/github/Billy67200/Advanced-Programming/blob/main/src/06_rag_llamaindex_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LlamaIndex RAG with Gemini: A Step-by-Step Tutorial


In this tutorial, we will walk through how to use [LlamaIndex](https://www.llamaindex.ai/) with Google's [Gemini models](https://ai.google.dev/models/gemini) to query information from a document. LlamaIndex is a powerful framework for building applications with large language models over your data. Gemini models are state-of-the-art language models from Google AI. By combining these tools, we can create intelligent applications that can understand and answer questions based on your documents.

Let's get started!

## Prerequisites

Before we begin, make sure you have the following:

1.  **Google Cloud Account and API Key:** You will need a Google Cloud account and an API key to access Gemini models. You can obtain an API key from the [Google AI Studio](https://makersuite.google.com/).
2.  **Google Colab or Python Environment:** This tutorial is designed to be run in Google Colab for ease of setup, but you can also adapt it to your local Python environment.

## Setup

First, we need to install the necessary Python packages. We will use `pip` to install `llama-index`, `llama-index-llms-gemini`, `llama-index-embeddings-gemini`, `google-generativeai`.

In [ ]:
%pip install llama-index-llms-gemini
!pip install -q llama-index google-generativeai

This command installs:

*   `llama-index-llms-gemini`:  LlamaIndex integration with Gemini language models.
*   `llama-index`: The core LlamaIndex library.
*   `google-generativeai`: Google's Python library for generative AI models, which is used under the hood by `llama-index-llms-gemini`.
    The `-q` flag in `!pip install -q` is for "quiet" mode, which suppresses the output during installation, making the notebook cleaner.

Next, install the Gemini embedding integration for LlamaIndex:

In [ ]:
pip install llama-index-embeddings-gemini

This installs:

*   `llama-index-embeddings-gemini`: LlamaIndex integration with Gemini embedding models.

Finally, install the core `llama-index` package if you haven't already. While often installed as a dependency, it's good practice to ensure it's explicitly installed.

In [ ]:
!pip install llama-index

Now, let's import the required modules from the installed libraries.

In [7]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.core import Settings
from google.colab import userdata

Here's a breakdown of the imports:

*   `VectorStoreIndex`, `SimpleDirectoryReader` from `llama_index.core`: These are core LlamaIndex classes. `SimpleDirectoryReader` is used to load documents from a directory or files, and `VectorStoreIndex` is used to create an index over your documents for efficient querying.
*   `Gemini` from `llama_index.llms.gemini`: This class allows us to use Gemini models as the language model within LlamaIndex.
*   `GeminiEmbedding` from `llama_index.embeddings.gemini`: This class allows us to use Gemini models for generating embeddings for our documents. Embeddings are numerical representations of text that capture their semantic meaning, crucial for similarity searches.
*   `Settings` from `llama_index.core`: This class is used to configure global settings for LlamaIndex, such as setting the LLM and embedding model to be used throughout the application.
*   `userdata` from `google.colab`: This is specific to Google Colab and is used to securely access user-stored data like API keys without hardcoding them in the notebook.

Before loading our data, let's download a sample document. We will use Paul Graham's essay "What I Worked On Before College" as our example document.

In [8]:
!wget https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt

--2025-03-06 08:47:30--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘paul_graham_essay.txt.1’

paul_graham_essay.t 100%[===================>]  73.28K  --.-KB/s    in 0.05s   

2025-03-06 08:47:30 (1.45 MB/s) - ‘paul_graham_essay.txt.1’ saved [75042/75042]



This command uses `wget` to download the text file directly from the LlamaIndex GitHub repository and saves it as `paul_graham_essay.txt` in your Colab environment.

Now, we need to set up our API key. For security and best practices, especially in environments like Colab, it's recommended to use `userdata.get` to retrieve your API key.  Make sure you have stored your Google API key in Colab's user secrets under the key name `GOOGLE_API_KEY`. You can do this by clicking on the "Secrets" icon (key icon) in the left sidebar of your Colab notebook.

In [9]:
api_key = userdata.get('GOOGLE_API_KEY')

This line retrieves your Google API key, which will be used to authenticate your requests to the Gemini models.

## Loading Data

Now we will load the Paul Graham essay that we downloaded. We use `SimpleDirectoryReader` to read the text file.

In [10]:
# Load data
reader = SimpleDirectoryReader(input_files=["paul_graham_essay.txt"])
documents = reader.load_data()

Here's what's happening:

*   `SimpleDirectoryReader(input_files=["paul_graham_essay.txt"])`: We initialize a `SimpleDirectoryReader` object, telling it to read from the file `paul_graham_essay.txt`.  Even though it's called `DirectoryReader`, it can also read single files.
*   `documents = reader.load_data()`: We call the `load_data()` method of the reader. This reads the content of the specified file and loads it into a list of `Document` objects. LlamaIndex `Document` is a container for text data and associated metadata.

## Configuring LLM and Embedding Model

Next, we need to configure which Gemini models LlamaIndex should use for language modeling (LLM) and embeddings.

In [11]:
# Define LLM and embedding model
llm = Gemini(api_key=api_key, model="models/gemini-2.0-flash-exp")
embed_model = GeminiEmbedding(model_name="models/text-embedding-004", api_key=api_key)
Settings.llm = llm
Settings.embed_model = embed_model

Let's break this down:

*   `llm = Gemini(api_key=api_key, model="models/gemini-2.0-flash-exp")`: We initialize a `Gemini` language model.
    *   `api_key=api_key`: We pass the API key we retrieved earlier to authenticate our requests.
    *   `model="models/gemini-2.0-flash-exp"`: We specify the Gemini model to use. Here, we are using `"models/gemini-2.0-flash-exp"`. You can explore other available Gemini models on the [Google AI Studio](https://makersuite.google.com/app/apikey).
*   `embed_model = GeminiEmbedding(model_name="models/text-embedding-004", api_key=api_key)`: We initialize a `GeminiEmbedding` model.
    *   `model_name="models/text-embedding-004"`: We specify the Gemini embedding model to use, `"models/text-embedding-004"`.  This is a model designed for generating text embeddings.
    *   `api_key=api_key`: Again, we pass the API key for authentication.
*   `Settings.llm = llm`: We set the global LLM for LlamaIndex to be the `Gemini` model we just initialized. This means that any LlamaIndex components that need an LLM will use this Gemini model by default.
*   `Settings.embed_model = embed_model`: Similarly, we set the global embedding model to be the `GeminiEmbedding` model. This ensures that LlamaIndex uses this model for generating embeddings.

By setting these in `Settings`, we make it easy to use these models throughout our LlamaIndex application without needing to specify them every time.

## Creating a Vector Store Index

Now we will create a vector store index from our loaded documents. A vector store index is a data structure that allows for efficient similarity search over document embeddings.

In [12]:
# Create vector store index
index = VectorStoreIndex.from_documents(documents)

*   `VectorStoreIndex.from_documents(documents)`: This line creates a `VectorStoreIndex` directly from the list of `Document` objects we loaded earlier. Under the hood, LlamaIndex will:
    1.  **Chunk the documents:** Break the documents into smaller text chunks if they are too long.
    2.  **Generate embeddings:** Use the `embed_model` (Gemini Embedding model we set in `Settings`) to generate embeddings for each text chunk.
    3.  **Store embeddings in an index:** Store these embeddings in a vector database (by default, a simple in-memory vector store is used). This index allows for fast retrieval of chunks that are semantically similar to a query.

## Querying the Index - Example 1

Let's ask a simple question about the document. We will create a query engine and then use it to query the index.

In [13]:
# Create query engine
query_engine = index.as_query_engine(similarity_top_k=2)
response = query_engine.query(
    "What were the two main things the author worked on before college?"
)
print(str(response))

Before college, the author spent most of his time writing and programming.



Here's what's happening:

*   `query_engine = index.as_query_engine(similarity_top_k=2)`: We create a `QueryEngine` from our `VectorStoreIndex`.
    *   `similarity_top_k=2`: This parameter tells the query engine to retrieve the top 2 most similar text chunks from the index for each query.  This is a parameter that can be tuned; a higher `k` might give more context to the LLM, but could also include less relevant information.
*   `response = query_engine.query("What were the two main things the author worked on before college?")`: We use the `query_engine` to ask our question.
    *   The query engine will first:
        1.  **Generate an embedding for the query:** Using the `embed_model`.
        2.  **Search the index:** Find the top `k=2` most similar text chunks in the vector store based on embedding similarity.
        3.  **Pass to LLM:**  Send these retrieved text chunks along with the query to the LLM (the `Gemini` model we set in `Settings`).
        4.  **Generate response:** The LLM generates an answer based on the context provided by the retrieved chunks and the original query.
*   `print(str(response))`: We print the response from the query engine.  `response` object contains the answer text, source documents, and other metadata. `str(response)` gives us the plain text answer.

You should see an answer from the Gemini model based on Paul Graham's essay, answering the question about what he worked on before college.

## Querying the Index - Example 2 (More Complex Question)

Let's try a more complex, multi-faceted question to test the capabilities of our setup.

In [14]:
q = "What were the key factors and accidental discoveries that led to the innovative approach of Y Combinator in funding and supporting startups, according to Paul Graham?"

Here we define a more detailed question `q` that asks about the factors behind Y Combinator's innovative approach, specifically asking for Paul Graham's perspective.

In [15]:
# Create query engine
query_engine = index.as_query_engine(similarity_top_k=2)
response = query_engine.query(
    q
)
print(str(response))

According to Paul Graham, Y Combinator's innovative approach was born out of ignorance about how angel investing was typically done. They combined the concepts of VC firms and angels, offering smaller investments with more help for founders. Not knowing how VC firms were organized, they funded YC with their own money, allowing them to keep all the returns. The batch model, a distinctive feature of YC, was discovered by accident due to their need to gain experience as investors. This led to the creation of the Summer Founders Program, where they could practice investing on "fake founders."



This code block is similar to the previous query example. We are reusing the same `query_engine` we created earlier (which is fine, query engines are reusable). We then use `query_engine.query(q)` to ask our more complex question `q`.

Again, the query engine will perform the same steps: embed the query, find similar text chunks, and pass them to the Gemini LLM to generate an answer.

The output should be a more detailed answer reflecting the complexity of the question, drawing information from the Paul Graham essay to address the query about Y Combinator's approach.

## Conclusion

Congratulations! You have successfully built a simple question-answering system using LlamaIndex and Gemini models. You loaded a document, created a vector index, and queried it with both simple and complex questions.

This tutorial provides a basic introduction. You can explore further by:

*   **Trying different Gemini models:** Experiment with other available Gemini models for both LLM and embedding to see how performance and response style vary.
*   **Adjusting `similarity_top_k`:**  See how changing the number of retrieved chunks affects the answers.
*   **Using different data:** Try loading your own documents and querying them.
*   **Exploring more advanced LlamaIndex features:** LlamaIndex offers many more features, such as different types of indexes, query modes, data connectors, and agentic capabilities. Dive deeper into the [LlamaIndex documentation](https://docs.llamaindex.ai/) to learn more.

This combination of LlamaIndex and Gemini provides a powerful and accessible way to build intelligent applications that can interact with and understand your data.